In [1]:
import numpy as np
import pandas as pd
import glob
import sys
import h5py
#from netCDF4 import Dataset
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import pyarrow as pa
import pyarrow.parquet as pq

from functools import reduce
import operator
import gc

In [2]:
# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

In [3]:
# PySpark packages
from pyspark import SparkContext   
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

spark = SparkSession.builder \
    .master("yarn") \
    .appName("spark-shell") \
    .config("spark.driver.maxResultSize", "32g") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "14g") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "60") \
    .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir("hdfs://sohnic:54310/tmp/checkpoints")

spark.conf.set("spark.sql.debug.maxToStringFields", 500)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/28 08:43:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/28 08:43:01 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
sc.getConf().getAll()[:10]

[('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://node6:8088/proxy/application_1701156349867_0003'),
 ('spark.driver.memory', '32g'),
 ('spark.app.submitTime', '1701160980300'),
 ('spark.executorEnv.PYTHONPATH',
  '/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/python/:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.9.7-src.zip'),
 ('spark.driver.maxResultSize', '32g'),
 ('spark.app.id', 'application_1701156349867_0003'),
 ('spark.master', 'yarn'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', 'sohnic'),
 ('spark.submit.deployMode', 'client')]

In [5]:
h5dir = '/mnt/raid5/lshin/'

In [6]:
flist = !ls /mnt/raid5/lshin

In [7]:
flist

['snap_099.0.0.0.hdf5']

In [8]:
h5list = flist#[2:]

In [9]:
numh5list = len(h5list)
print(numh5list)

1


In [10]:
h5dir+h5list[0]

'/mnt/raid5/lshin/snap_099.0.0.0.hdf5'

In [11]:
h5py.is_hdf5(h5dir+h5list[0])

True

In [12]:
try:
    h5f = h5py.File(h5dir+h5list[0], "r")
except IOError as e:
    print("Error opening HDF5 file:", str(e))
# Don't forget f.close() when done! 

In [13]:
h5f.keys()

<KeysViewHDF5 ['PartType0', 'PartType1', 'PartType4', 'PartType5']>

In [14]:
for key in h5f.keys():
    item = h5f[key]
    
    if isinstance(item, h5py.Group):  # Check if it's a group
        print(f"Group: {key}")
    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
        print(f"Dataset: {key}, dtype: {item.dtype}")
    else:
        print(f"Unknown item: {key}")

Group: PartType0
Group: PartType1
Group: PartType4
Group: PartType5


In [15]:
h5f['PartType4'].keys()

<KeysViewHDF5 ['Coordinates', 'GFM_StellarFormationTime', 'Masses']>

In [16]:
for key in h5f['PartType4'].keys():
    item = h5f['PartType4'][key]
    
    if isinstance(item, h5py.Group):  # Check if it's a group
        print(f"Group: {key}")
    elif isinstance(item, h5py.Dataset):  # Check if it's a dataset
        print(f"Dataset: {key}, dtype: {item.dtype}, shape: {item.shape}")
    else:
        print(f"Unknown item: {key}")

Dataset: Coordinates, dtype: float32, shape: (206055, 3)
Dataset: GFM_StellarFormationTime, dtype: float32, shape: (206055,)
Dataset: Masses, dtype: float32, shape: (206055,)


# Save selected features as a parquet

In [17]:
#schema = T.StructType([\
##                       T.StructField('filename',T.StringType(), True),\
#                       T.StructField('gas_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('gas_mass',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('dm_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('star_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('star_form_time',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('star_mass',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('bh_coord',T.ArrayType(T.FloatType(),True), True),\
#                       T.StructField('bh_mass',T.ArrayType(T.FloatType(),True), True),\
#                   ])
schema = T.StructType([\
#                       T.StructField('filename',T.StringType(), True),\
                       T.StructField('star_coord_x',T.FloatType(), True),\
                       T.StructField('star_coord_y',T.FloatType(), True),\
                       T.StructField('star_coord_z',T.FloatType(), True),\
#                       T.StructField('gas_mass',T.FloatType(), True),\
#                       T.StructField('dm_coord',T.FloatType(), True),\
#                       T.StructField('star_coord',T.FloatType(), True),\
#                       T.StructField('star_form_time',T.FloatType(), True),\
#                       T.StructField('star_mass',T.FloatType(), True),\
#                       T.StructField('bh_coord',T.FloatType(), True),\
#                       T.StructField('bh_mass',T.FloatType(), True),\
                      ])

In [18]:
h5f['PartType4/Coordinates'].shape

(206055, 3)

In [19]:
h5f['PartType4/Coordinates'][:5]

array([[1315.3317, 4634.5044, 1751.5077],
       [1315.0211, 4634.4673, 1751.346 ],
       [1314.794 , 4634.2354, 1751.6212],
       [1315.2739, 4634.2817, 1751.7683],
       [1315.1174, 4634.261 , 1751.265 ]], dtype=float32)

In [20]:
star_coord_x = h5f['PartType4/Coordinates'][:,0].tolist()
star_coord_y = h5f['PartType4/Coordinates'][:,1].tolist()
star_coord_z = h5f['PartType4/Coordinates'][:,2].tolist()

In [23]:
#gas_coord = h5f['PartType0/Coordinates'][:,:].flatten().tolist()
#gas_mass =  [float(i) for i in h5f['PartType0/Masses']]
#dm_coord = h5f['PartType1/Coordinates'][:,:].flatten().tolist()
#star_coord = h5f['PartType4/Coordinates'][:,:].flatten().tolist()
#star_form_time =  [float(i) for i in h5f['PartType4/GFM_StellarFormationTime']]
#star_mass =  [float(i) for i in h5f['PartType4/Masses']]
#bh_coord = h5f['PartType5/Coordinates'][:,:].flatten().tolist()
#bh_mass = [float(i) for i in h5f['PartType5/Masses']]

In [24]:
%%time
sparkdf = spark.createDataFrame(zip(star_coord_x, star_coord_y, star_coord_z),schema)
#sparkdf = spark.createDataFrame(zip(gas_coord_x),schema)

CPU times: user 438 ms, sys: 9.2 ms, total: 448 ms
Wall time: 1.97 s


In [25]:
outdir = 'hdfs://sohnic:54310/user/lshin/'

In [26]:
outname = outdir+h5list[0].replace("hdf5","parquet.snappy")
print(outname)

hdfs://sohnic:54310/user/lshin/snap_099.0.0.0.parquet.snappy


In [27]:
%%time
sparkdf.write.option("compression", "snappy") \
    .mode("overwrite") \
    .save(outname)

CPU times: user 31.9 ms, sys: 2.65 ms, total: 34.6 ms
Wall time: 1min 18s


# Check up the parquet

In [28]:
sparkdf.printSchema()

root
 |-- star_coord_x: float (nullable = true)
 |-- star_coord_y: float (nullable = true)
 |-- star_coord_z: float (nullable = true)



In [29]:
%%time
print(sparkdf.count())

23/11/28 08:45:59 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 240) (node6 executor 28): FetchFailed(BlockManagerId(33, node2, 41709, None), shuffleId=0, mapIndex=28, mapId=148, reduceId=0, message=
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:437)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:1232)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:971)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:86)
	at org.apache.spark.util.CompletionIterator.next(CompletionIterator.scala:29)
	at scala.collection.Iterator$$anon$10.nextCur(Iterator.scala:587)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:601)
	at scala.collection.Iterator$$anon$9.hasNext(Iterator.scala:576)
	at org.apache.spark.util.CompletionIterator.hasNext(Co

Py4JJavaError: An error occurred while calling o137.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 3 (count at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason:
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:437)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:1232)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:971)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:86)
	at org.apache.spark.util.CompletionIterator.next(CompletionIterator.scala:29)
	at scala.collection.Iterator$$anon$10.nextCur(Iterator.scala:587)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:601)
	at scala.collection.Iterator$$anon$9.hasNext(Iterator.scala:5

In [30]:
%%time
sparkdf.limit(2).toPandas().transpose()

CPU times: user 68.9 ms, sys: 4.92 ms, total: 73.9 ms
Wall time: 15.7 s


,0,1
star_coord_x,19096.705078,19092.337891
star_coord_y,1555.820557,1552.883545
star_coord_z,19117.621094,19122.544922


In [31]:
sparkdf.select(['star_coord_x','star_coord_y','star_coord_z']).show(3)
#sparkdf.select(['star_coord_x']).show(3)

+------------+------------+------------+
|star_coord_x|star_coord_y|star_coord_z|
+------------+------------+------------+
|   1315.3317|   4634.5044|   1751.5077|
|   1315.0211|   4634.4673|    1751.346|
|    1314.794|   4634.2354|   1751.6212|
+------------+------------+------------+
only showing top 3 rows



In [32]:
%%time
sparkdf.select(['star_coord_x','star_coord_y','star_coord_z']).describe().toPandas().set_index('summary').transpose()
#sparkdf.select(['gas_coord_x']).describe().toPandas().set_index('summary').transpose()

23/11/28 08:48:18 WARN TaskSetManager: Lost task 0.0 in stage 9.0 (TID 492) (node5 executor 38): FetchFailed(BlockManagerId(31, node1, 44335, None), shuffleId=2, mapIndex=56, mapId=428, reduceId=0, message=
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:437)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:1232)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:971)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:86)
	at org.apache.spark.util.CompletionIterator.next(CompletionIterator.scala:29)
	at scala.collection.Iterator$$anon$10.nextCur(Iterator.scala:587)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:601)
	at scala.collection.Iterator$$anon$9.hasNext(Iterator.scala:576)
	at org.apache.spark.util.CompletionIterator.hasNext(Co

Py4JJavaError: An error occurred while calling o175.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:98)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:94)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.C